In [9]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,os,time,datetime,sys

import torch
import pandas as pd

import gym

In [10]:
class Net(torch.nn.Module):
    def __init__(self,env):
        super(Net,self).__init__()
        self.l0 = torch.nn.Linear(env.observation_space.shape[0],16)
        self.l1 = torch.nn.Linear(16,env.action_space.n)
    def forward(self,x):
        x = torch.nn.functional.relu(self.l0(x))
        x = torch.nn.functional.softmax(self.l1(x))
        return x

In [11]:
def discount_rewards(rewards,gamma=0.99):
    r = np.array([gamma**k*rewards[k] for k in range(len(rewards))])
    r = r[::-1].cumsum()[::-1]
    return r-r.mean()

In [12]:
batch_size = 25

In [13]:
env = gym.make('CartPole-v0')
action_space = np.arange(env.action_space.n)

net = Net(env)
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)

C:\Users\Jan\Anaconda3\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [14]:
total_rewards = []

batch_rewards = []
batch_actions = []
batch_states = []

batch_counter = 1

episodes = 0
while episodes<500:

    s0 = env.reset()

    states = []
    rewards = []
    actions = []

    done = False

    while not done:

        action_probs = net(torch.FloatTensor(s0)).detach().numpy()
        action_c = rnd.choice(action_space,p=action_probs)

        s1,r,done,_ = env.step(action_c)

        states.append(s0)
        rewards.append(r)
        actions.append(action_c)

        s0 = s1

        if done:

            total_rewards.append(sum(rewards))

            batch_states.extend(states)
            batch_rewards.extend(discount_rewards(rewards))
            batch_actions.extend(actions)

            batch_counter += 1

            if batch_counter==batch_size:

                optimizer.zero_grad()

                state_t = torch.FloatTensor(batch_states)
                reward_t = torch.FloatTensor(batch_rewards)
                action_t = torch.LongTensor(batch_actions)

                logprob = torch.log(net(state_t))
                selected_logprobs = reward_t*torch.gather(logprob,1,action_t.reshape(action_t.shape[0],1)).squeeze()
                loss = -selected_logprobs.mean()

                loss.backward()
                optimizer.step()

                batch_states = []
                batch_rewards = []
                batch_actions = []
                batch_counter = 1
                
            print('Avg rewards over previous 100 episodes: {}'.format(np.mean(total_rewards[-100:])))
            episodes += 1
            
env.close()

C:\Users\Jan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


Avg rewards over previous 100 episodes: 8.0
Avg rewards over previous 100 episodes: 14.0
Avg rewards over previous 100 episodes: 18.0
Avg rewards over previous 100 episodes: 16.75
Avg rewards over previous 100 episodes: 17.6
Avg rewards over previous 100 episodes: 16.5
Avg rewards over previous 100 episodes: 16.0
Avg rewards over previous 100 episodes: 15.5
Avg rewards over previous 100 episodes: 16.555555555555557
Avg rewards over previous 100 episodes: 17.4
Avg rewards over previous 100 episodes: 18.0
Avg rewards over previous 100 episodes: 17.75
Avg rewards over previous 100 episodes: 17.076923076923077
Avg rewards over previous 100 episodes: 17.0
Avg rewards over previous 100 episodes: 17.2
Avg rewards over previous 100 episodes: 16.875
Avg rewards over previous 100 episodes: 17.352941176470587
Avg rewards over previous 100 episodes: 17.055555555555557
Avg rewards over previous 100 episodes: 16.842105263157894
Avg rewards over previous 100 episodes: 16.9
Avg rewards over previous 1

Avg rewards over previous 100 episodes: 29.75
Avg rewards over previous 100 episodes: 29.59
Avg rewards over previous 100 episodes: 29.77
Avg rewards over previous 100 episodes: 29.75
Avg rewards over previous 100 episodes: 29.83
Avg rewards over previous 100 episodes: 29.76
Avg rewards over previous 100 episodes: 29.58
Avg rewards over previous 100 episodes: 29.46
Avg rewards over previous 100 episodes: 30.08
Avg rewards over previous 100 episodes: 30.3
Avg rewards over previous 100 episodes: 31.31
Avg rewards over previous 100 episodes: 31.56
Avg rewards over previous 100 episodes: 31.65
Avg rewards over previous 100 episodes: 31.8
Avg rewards over previous 100 episodes: 31.84
Avg rewards over previous 100 episodes: 31.41
Avg rewards over previous 100 episodes: 31.46
Avg rewards over previous 100 episodes: 31.3
Avg rewards over previous 100 episodes: 31.05
Avg rewards over previous 100 episodes: 31.15
Avg rewards over previous 100 episodes: 31.21
Avg rewards over previous 100 episode

Avg rewards over previous 100 episodes: 41.19
Avg rewards over previous 100 episodes: 41.34
Avg rewards over previous 100 episodes: 41.53
Avg rewards over previous 100 episodes: 41.65
Avg rewards over previous 100 episodes: 41.73
Avg rewards over previous 100 episodes: 42.01
Avg rewards over previous 100 episodes: 42.15
Avg rewards over previous 100 episodes: 41.98
Avg rewards over previous 100 episodes: 41.89
Avg rewards over previous 100 episodes: 41.58
Avg rewards over previous 100 episodes: 41.85
Avg rewards over previous 100 episodes: 41.75
Avg rewards over previous 100 episodes: 41.75
Avg rewards over previous 100 episodes: 42.17
Avg rewards over previous 100 episodes: 42.52
Avg rewards over previous 100 episodes: 42.71
Avg rewards over previous 100 episodes: 42.22
Avg rewards over previous 100 episodes: 42.05
Avg rewards over previous 100 episodes: 42.28
Avg rewards over previous 100 episodes: 42.46
Avg rewards over previous 100 episodes: 42.72
Avg rewards over previous 100 epis

In [15]:
env = gym.make('CartPole-v0')

s0 = env.reset()
env.render()

done = False
while not done:
    
    action_probs = net(torch.FloatTensor(s0)).detach().numpy()
    action_c = rnd.choice(action_space,p=action_probs)
    
    s0,r,done,_ = env.step(action_c)
    env.render()

env.close()

C:\Users\Jan\Anaconda3\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


C:\Users\Jan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
